# Optimization Algorithms


Scikit-Learn LogisticRegression model uses various optimization algorithms for the Gradient Descent algorithm.

In this notebook, we try to understand how to use some of the optimization algorithms.



## SciPy Optimization 

The scipy.optimize package provides several commonly used optimization algorithms. 
https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

We will study the **unconstrained minimization of multivariate scalar functions (minimize)** algorithms.
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

The commonly used "minimize" algorithms are:
- CG (nonlinear conjugate gradient)
- Newton-CG (truncated Newton)
- TNC (truncated Newton algorithm)
- BFGS (Broyden-Fletcher-Goldfarb-Shanno)
- L-BFGS-B (bound constrained minimization)
- Nelder-Mead
- Powell


## How do We Use the scipy.optimize.minimize Function?


##### scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)

We will use the folowng arguments:
- 1st (fun)
- 2nd (x0)
- 3rd (args)
- 4th (method)
- 5th (jac)
- 11th (options)


See the description below:

- 1st(fun):
callable
The objective function to be minimized. Must be in the form f(x, *args). The optimizing argument, x, is a 1-D array of points, and args is a tuple of any additional fixed parameters needed to completely specify the function.

- 2nd (x0):
ndarray
Initial guess. 
len(x0) is the dimensionality of the minimization problem.


- 3rd (args): 
tuple, optional
Extra arguments passed to the objective function and its derivatives (Jacobian, Hessian).

- 4th (method):
str or callable, optional

Type of solver. Should be one of

    ‘Nelder-Mead’ 
    ‘Powell’ 
    ‘CG’ 
    ‘BFGS’ 
    ‘Newton-CG’ 
    ‘L-BFGS-B’ 
    ‘TNC’ 
    ‘COBYLA’ 
    ‘SLSQP’ 
    ‘dogleg’ 
    ‘trust-ncg’ 
    ‘trust-exact’ 
    ‘trust-krylov’ 


If not given, chosen to be one of BFGS, L-BFGS-B, SLSQP, depending if the problem has constraints or bounds.


- 5th (jac):
bool or callable, optional

Jacobian (gradient) of objective function. Only for CG, BFGS, Newton-CG, L-BFGS-B, TNC, SLSQP, dogleg, trust-ncg, trust-krylov, trust-region-exact. If jac is a Boolean and is True, fun is assumed to return the gradient along with the objective function. If False, the gradient will be estimated numerically. jac can also be a callable returning the gradient of the objective. In this case, it must accept the same arguments as fun.

- 11th (options):
dict, optional

A dictionary of solver options. All methods accept the following generic options:

- maxiter : int
Maximum number of iterations to perform.

- disp : bool
Set to True to print convergence messages.

There are other method-specific options.


We will see **<font color=red size=4> how to use the BFGS algorithm </font>** by solving three unconstrained optimization problems.


## BFGS

For the BFGS solver, the following options are available:
	
- disp : bool (Set to True to print convergence messages.)
- maxiter : int (Maximum number of iterations to perform.)
- gtol : float (Gradient norm must be less than gtol before successful termination.)
- norm : float [Order of norm (Inf is max, -Inf is min).]
- eps : float or ndarray (If jac is approximated, use this value for the step size.)

The two most relevant "options" for BFGS are: disp and maxiter



## Return Values from the "minimize" dunction
	
res : OptimizeResult

The optimization result represented as a OptimizeResult object. 

Important attributes are: 
- x: the solution array
- success: a Boolean flag indicating if the optimizer exited successfully and message which describes the cause of the termination. 


In [1]:
from scipy import optimize
import numpy as np
from numpy import linalg as LA

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [2]:
#help(optimize)

## Minimization Problem 1

Minimize the following objective function:

$f(x) = 4x^2 - 64x$

Find the x that minimizes f(x).

In [3]:
# Objectve function to be minimized
def f(x):
    return 4*x**2 - 64*x


# Gradient of the objective function
def df(x):
    return 8*x - 64


# Initial value of x
x0 = 1

# Try with BFGS
result = optimize.minimize(f,x0,method='bfgs',jac=df,options={'disp':True, 'maxiter':20})
print("\nSolution: ", result.x)
print("Optimizer exits successfully: ", result.success)

Optimization terminated successfully.
         Current function value: -256.000000
         Iterations: 3
         Function evaluations: 4
         Gradient evaluations: 4

Solution:  [8.]
Optimizer exits successfully:  True


## Minimization Problem 2

Minimize a general quadratic objective function:

A general purely quadratic real function f(x) on $n$ real variables $x_1, ..., x_n$ can always be written as $x^TAx$ where x is the column vector with those variables, and A is a symmetric real matrix ($A = A^T$). 

$f(x) = x^TAx$

Therefore, the matrix (A) being **positive definite** means that f has a **unique minimum (zero)** when x is zero, and is strictly positive for any other x.

The determinant of a positive definite matrix is always positive.

In [4]:
# A positive definite matrix to be used in the function definitions
A = np.array([[2.,2.],[2.,3.]])

print("Matrix A:")
print(A)

eigenvalues, eigenvectors = LA.eig(A)
print("Eigenvalues of A: ", eigenvalues)

# Objectve function to be minimized: f(x) = x^T A x
def f(x):
    return np.dot(x.T,np.dot(A,x))

# Gradient of the objective function, df = 2*A*x
def df(x):
    return 2*np.dot(A,x)

# Initial value of x
x_initial = np.array([1.,2.])

# Try with BFGS
result = optimize.minimize(fun = f, x0 = x_initial, method = 'bfgs',jac = df,options = {'disp': True, 'maxiter': 20})

print("\nSolution: ", result.x)
print("Optimizer exits successfully: ", result.success)

Matrix A:
[[2. 2.]
 [2. 3.]]
Eigenvalues of A:  [0.43844719 4.56155281]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 5

Solution:  [-1.60761179e-07 -1.97082922e-07]
Optimizer exits successfully:  True


## Minimization Problem 3 (Binary Classification using Logistic Regression)

Now we will solve a classification problem using Logistic Regression.

The goal is to minimize an objective function (cost function) to find optimal model parameters (theta).

We will use a synthetic data set that has 4 training examples belonging to 2 classes.

In [5]:
# Demo: A simple synthetic dataset is used to understand the implementation of the optimize.minimize solvers 
#       to find the optimal theta


# Note: To use the SciPy minimize function, the objective function must be in the form f(x, *args). 
# The optimizing argument, x, is a 1-D array of points, 
#     and args is a tuple of any additional fixed parameters needed to completely specify the function.
# In the following "cost" and "grad" are functions.
# The first arguments of these two functions should be "theta" (optimization argument)


# Sigmoid function
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))


# Cost (cross-entropy) function
def cost(theta, X, y):
    N = X.shape[0]
    d = X.shape[1]
   
    # Reshape theta to make it a 1D (n x 1) column vector
    # This is important, because "X" is a N x d matrix.
    # Hence, the dot product cannot be done unless "theta" is d x 1
    
    theta = theta.reshape((d,1))
   
    h = sigmoid(X.dot(theta))
    
    J = -(1/N)*np.sum((y*np.log(h) + (1 - y)*np.log(1 - h)))
    return J



# Gradient of the cost function
def grad(theta, X, y):
    N = X.shape[0]
    d = X.shape[1]
    
    # Reshape theta to make it a 1D (d x 1) column vector
    # This is important, because "X" is a N x d matrix.
    # Hence, the dot product cannot be done unless "theta" is d x 1
    theta = theta.reshape((d,1));
    
    h = sigmoid(X.dot(theta))
   
    grad = (1/N)*(X.T.dot(h - y))
    
    return grad.flatten()
  

# Create the data matrix X and target vector y
X = np.array([[10, 20, 30],[40, 1, 5], [12, 22, 32],[38, 3, 4]])
y = np.array([[1], [0], [1], [0]])

print("X:\n",X)
print("y:\n",y)



# Initialize theta
theta = np.zeros(X.shape[1]);


result = optimize.minimize(fun = cost, x0 = theta, args =(X, y), 
                           method='bfgs', jac = grad, options={'disp': True, 'maxiter': 100})


optimal_theta = result.x;

print("\nOptimal Theta: ", optimal_theta)
print("Optimizer exits successfully: ", result.success)



# Predict class probabilities
y_proba = sigmoid(X.dot(optimal_theta))


# Predict the class labels using class probabilities
y_predicted = np.zeros((len(y))).reshape(len(y), 1)

for i in range(len(y)):
    if(y_proba[i] >= 0.5):
        y_predicted[i] = 1
    else:
        y_predicted[i] = 0
        
        
print("\nConfusion Matrix:")
print(confusion_matrix(y, y_predicted))


print("\n\nScikit-Learn Logistic Regression\n")

# Classification using sklearn's Logistic Regression


lg_reg_clf = LogisticRegression(solver="lbfgs")

y = y.ravel() # LogisticRegression object requires a flattened array, thus we apply the ravel() function

lg_reg_clf.fit(X, y)

print("No. of Iterations:", lg_reg_clf.n_iter_ )
print("\nWeight Intercept:", lg_reg_clf.coef_ )
print("Weight Coefficients:", lg_reg_clf.coef_ )

y_predicted = lg_reg_clf.predict(X)

print("\nScikit-Learn Confusion Matrix:")
print(confusion_matrix(y, y_predicted))

X:
 [[10 20 30]
 [40  1  5]
 [12 22 32]
 [38  3  4]]
y:
 [[1]
 [0]
 [1]
 [0]]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2

Optimal Theta:  [-0.65798587  0.44649041  0.62273663]
Optimizer exits successfully:  True

Confusion Matrix:
[[2 0]
 [0 2]]


Scikit-Learn Logistic Regression

No. of Iterations: [13]

Weight Intercept: [[-0.17375947  0.11807689  0.16751294]]
Weight Coefficients: [[-0.17375947  0.11807689  0.16751294]]

Scikit-Learn Confusion Matrix:
[[2 0]
 [0 2]]
